In [1]:
import pandas as pd
import os
from transformers import BertTokenizer
import seaborn as sns
import matplotlib.pyplot as plt

I1115 17:58:32.711898 4667254208 __init__.py:43] Loading faiss.


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',add_special_tokens=True,mask_token='@placeholder')

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [3]:
def replace_in_list(x):
    return ['[MASK]' if i=='@placeholder' else i for i in x]

In [11]:
def get_short_dataset(path,name):
    df=pd.read_json(path, lines=True)
    df['concat']= df['article']+df['question']
    ids_concat = df['concat'].apply((lambda x: tokenizer.tokenize(x)))
    label=df['label']
    option0 = df['option_0']
    option1 = df['option_1']
    option2 = df['option_2']
    option3 = df['option_3']
    option4 = df['option_4']
    ids_questions = df['question'].apply((lambda x: tokenizer.tokenize(x)))
    ids_context = df['article'].apply((lambda x: tokenizer.tokenize(x)))
    ids_concat = df['concat'].apply((lambda x: tokenizer.tokenize(x)))
    too_long=[i for i in range(len(ids_concat)) if(len(ids_concat[i]))>508]
    for i in too_long:
        ids_questions.pop(i)
        ids_context.pop(i)
        ids_concat.pop(i)
        label.pop(i)
        option0.pop(i)
        option1.pop(i)
        option2.pop(i)
        option3.pop(i)
        option4.pop(i)
        
    ids_questions = ids_questions.apply((lambda x: replace_in_list(x)))
    ids_concat = ids_concat.apply((lambda x: replace_in_list(x)))
    short_questions = ids_questions.apply((lambda x: tokenizer.convert_tokens_to_string(x)))
    short_context = ids_context.apply((lambda x: tokenizer.convert_tokens_to_string(x)))
    short_concat = ids_concat.apply((lambda x: tokenizer.convert_tokens_to_string(x)))
    d = {'questions': short_questions, 'context': short_context, 'concat': short_concat,'option_0':option0,
                'option_1':option1,'option_2':option2,'option_3':option3,'option_4':option4,'label':label}
    df_short = pd.DataFrame(d)
    df_short.to_csv('data/clean_training_data/{}.csv'.format(name))

In [12]:
get_short_dataset('data/training_data/Task_1_dev.jsonl','val')
get_short_dataset('data/training_data/Task_1_train.jsonl','train')
get_short_dataset('data/trail_data/Task_1_Imperceptibility.jsonl','test')

In [241]:
df=pd.read_json('training_data/Task_1_dev.jsonl', lines=True)
df['concat']= df['article']+df['question']
ids_concat = df['concat'].apply((lambda x: tokenizer.tokenize(x)))
options_label_cols={'label':df['label'],
                    'option0':df['option_0'],
                    'option1':df['option_1'],
                    'option2':df['option_2'],
                    'option3':df['option_3'],
                    'option4':df['option_4']}
ids_questions = train_1['question'].apply((lambda x: tokenizer.tokenize(x)))
ids_context = train_1['article'].apply((lambda x: tokenizer.tokenize(x)))
ids_concat = train_1['concat'].apply((lambda x: tokenizer.tokenize(x)))
too_long=[i for i in range(len(ids_concat)) if(len(ids_concat[i]))>508]
for i in too_long:
    ids_questions.pop(i)
    ids_context.pop(i)
    ids_concat.pop(i)
    for k in options_label_cols.keys():
        options_label_cols[k].pop(i)
tokenized_questions = ids_questions.apply((lambda x: tokenizer.encode(x)))
tokenized_context = ids_context.apply((lambda x: tokenizer.encode(x)))

In [125]:
tokenized_concat=[x+y[1:] for x,y in zip(tokenized_questions,tokenized_context)]

In [5]:
!python ./multiple-choice/run_tf_multiple_choice.py \
--task_name swag \
--mode sequence-classification \
--model_name_or_path bert-base-uncased \
--data_dir ./data/processed_data \
--do_train \
--do_eval \
--learning_rate 5e-5 \
--num_train_epochs 3 \
--max_seq_length 80 \
--per_gpu_eval_batch_size=16 \
--per_gpu_train_batch_size=16 \
--gradient_accumulation_steps 2 \
--eval_steps 100 \
--logging_steps 100 \
--evaluate_during_training \
--output_dir models_race4 \
--logging_dir ./runs \
--overwrite_output

I1111 10:24:26.506546 4675175872 __init__.py:43] Loading faiss.
//anaconda3/lib/python3.7/site-packages/transformers/training_args.py:339: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,
2020-11-11 10:24:27.219530: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-11-11 10:24:27.235183: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff236e3d130 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-11 10:24:27.235209: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
W1111 10:24:27.236079 4675175872 run_tf_multiple_choi

In [77]:
!help('modules')

/bin/bash: -c: line 0: syntax error near unexpected token `'modules''
/bin/bash: -c: line 0: `help('modules')'
